In [21]:

#PigLab4NoteBook
import pandas as pd

! pip install pandas
! pip install openpyxl
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [22]:
import pandas as pd

# Define the file paths
files = ['data/PigData/Lab4/Copia de Dades projecte DECIDE 20230724.csv']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

df = dfs[0]


In [23]:
df.head(50)

,Identificación,Fecha,Provincia,Patógeno,Resultado,Edad,Tipo Edad,Material,Animal,Categoría
0,1,01/07/2020,NaN,M. hyopneumoniae,NEG,0.0,NaN,T-Pulmón,NaN,NaN
1,1,01/07/2020,NaN,Influenza,NEG,0.0,NaN,T-Pulmón,NaN,NaN
2,2,01/07/2020,NaN,M. hyopneumoniae,POS,0.0,NaN,T-Pulmón,NaN,NaN
3,2,01/07/2020,NaN,Influenza,NEG,0.0,NaN,T-Pulmón,NaN,NaN
4,3,01/07/2020,NaN,M. hyopneumoniae,POS,0.0,NaN,T-Pulmón,NaN,NaN
5,3,01/07/2020,NaN,Influenza,POS,0.0,NaN,T-Pulmón,NaN,NaN
6,4,01/07/2020,NaN,M. hyopneumoniae,POS,0.0,NaN,T-Pulmón,NaN,NaN
7,4,01/07/2020,NaN,Influenza,NEG,0.0,NaN,T-Pulmón,NaN,NaN
8,5,01/07/2020,NaN,Influenza,NEG,0.0,NaN,F-Fluido Oral,NaN,NaN
9,5,01/07/2020,NaN,Influenza,POS,0.0,NaN,F-Fluido Oral,NaN,NaN


In [24]:

#Here Age, Ageunit and Farmtype are PigStage


df = df.rename(columns={
    'Identificación': 'SampleNumber',
    'Fecha': 'Date',
    'Provincia': 'Province',
    'Patógeno': 'Pathogen',
    'Resultado': 'Result',
    'Edad': 'Age', #Age of animal
    'Tipo Edad': 'AgeUnit', #represents the unit used to measure the age of animals (e.g, Weeks, Month,year)
    'Material': 'SampleType',
    'Animal': 'BreedType', 
    'Categoría':'FarmType',#refers to the productive stage (or the farming period) of the animals (fattening, nursery, weaned pigs, etc.)
})

df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "4"



# Combine 'Age' and 'Age Unit' columns into a new column 'Combined_Age'
df['Age'] = df['Age'].replace(0.0, 'Missing')
df['AgeUnit'] = df['AgeUnit'].fillna('Missing')

df['Age'] = df['Age'].astype(str) + ' ' + df['AgeUnit'].astype(str)

# Replace 'Missing' if both 'Age' and 'AgeUnit' are missing
df['Age'] = df.apply(lambda row: 'Missing' if row['Age'] == 'Missing Missing' else row['Age'], axis=1).fillna('Missing')


# Drop the original 'Edad' and 'Tipo Edad' columns if needed
df = df.drop(['AgeUnit'], axis=1)



In [25]:
#Annotation Mapping

#Pathogen and Sample Mapping and Annotations
PathogenMapping = {
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza" : "Swine influenza virus",
}

df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")

SampleTypeMapping = {
    'T-Pulmón': 'Tissue',
    'F-Fluido Oral': 'Fluid',
    'F-Líquido traqueobronquial': 'BAL',
    'T-Pulmón y bronquio': 'Mixed',
    'T-Pulmón y linfonodo': 'Mixed',
    'T-Pulmón y tonsila': 'Mixed',
    'Escobillones': 'Swab',
    'Tejidos': 'Tissue',
    'T-Pulmón y timo': 'Mixed',
    'Fluídos del procesado': 'Fluid',
    'F-Líquido traqueal': 'BAL',
    'T-Lenguas': 'Tissue',
    'E-Nasales': 'Swab',
    'T-Pulmón, linfonodo y tonsila': 'Mixed',
    'T-Testículos': 'Tissue',
    'T-Pulmón y corazón': 'Mixed',
    'E-Pulmón': 'Swab',
    'T-Pulmón,hígado,riñón,bazo,int': 'Mixed',
    'FTA': 'Swab',
    'T-Pulmón e hígado': 'Mixed',
    'E-Vaginales': 'Swab',
    'E-Escobillón de saliva': 'Swab',
    'T-Linfonodo': 'Tissue',
    'Fluídos corp.': 'Fluid',
    'E-Escobillón abdominal': 'Swab',
    'T-Pulmón y bazo': 'Mixed'
}

# Ex

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "NEG": "0",
    "POS": "1",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")


# Province annotation  mapping
ProvinceMapping = {
    'Girona': 'Girona',
    'Granada': 'Granada',
    'Huesca': 'Huesca',
    'Sevilla': 'Seville', #check on Tableau its work or not?
    'Ávila': 'Avila',
    'Lleida': 'Lleida',
    'Zaragoza': 'Zaragoza',
    'Huelva': 'Huelva',
    'Barcelona': 'Barcelona',
    'Segovia': 'Segovia',
    'Toledo': 'Toledo',
    'Málaga': 'Malaga',
    'Murcia': 'Murcia',
    'Badajoz': 'Badajoz',
    'Alicante': 'Alicante',
    'Pontevedra': 'Pontevedra',
    'Soria': 'Soria',
    'Salamanca': 'Salamanca',
    'Jaén': 'Jaen',
    'Albacete': 'Albacete',
    'Zamora': 'Zamora',
    'Almería': 'Almeria',
    'Tarragona': 'Tarragona',
    'Valladolid': 'Valladolid',
    'Madrid': 'Madrid',
    'Valencia': 'Valencia',
    'Navarra': 'Navarre',
    'Castellón': 'Castellon',
    'Orense': 'Ourense',
    'Burgos': 'Burgos',
    'Palencia': 'Palencia',
    'La Coruña': 'ACoruna',
    'nan': 'Missing'  # Handle missing values
}


# Translate provinces in the 'referncia' column using the mapping
df['Province'] = df['Province'].map(ProvinceMapping).fillna('Missing')

# Step 1: Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Step 2: Format to Year, Month, Day
df['Date'] = df['Date'].dt.strftime("%Y-%m-%d")




In [26]:
FarmTypeMapping = {
    "Final Transición": "Finishing",
    "final Transición": "Finishing",
    " Final Transición": "Finishing",
    "Transición": "Finishing",
    "nan": "Missing",
    "Engorde": "Finishing",
    "Primíparas": "Sow",
    "Multíparas": "Sow",
    "Nulíparas": "Sow",
    "Destetados": "'Weaning",
    "Fut. Reproduct.": "Sow",
    "Reproductoras": "Sow",
    "Gestación": "Sow",
    "Lactación": "Sow",
    "Reposición": "Gilt",

}
df["FarmType"] = df["FarmType"].map(FarmTypeMapping)

#BreedType Mapping

BreedTypeMapping = {
    'Cerdas': 'Sow',
    'cerdas': 'Sow',
    'Cerdos': 'Finishing',
    'cerdos': 'Finishing',
    'Lechones': 'Weaning',
    'lechones': 'Weaning',
    'Verracos': 'Boar',
    'verracos': 'Boar',
    'Cerdas jóvenes': 'Gilt',
    'cerdas jóvenes': 'Gilt',
    'nan': 'Missing',
    '': 'Missing',
}
df["BreedType"] = df["BreedType"].map(BreedTypeMapping)


In [27]:
df.head(50)

,SampleNumber,Date,Province,Pathogen,Result,Age,SampleType,BreedType,FarmType,DiagnosticTest,Country,Breed,Lab_Reference
0,1,2020-07-01,Missing,Mycoplasma hyopneumoniae,0,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
1,1,2020-07-01,Missing,Swine influenza virus,0,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
2,2,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
3,2,2020-07-01,Missing,Swine influenza virus,0,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
4,3,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
5,3,2020-07-01,Missing,Swine influenza virus,1,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
6,4,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
7,4,2020-07-01,Missing,Swine influenza virus,0,Missing,Tissue,NaN,NaN,PCR,Spain,Meat,4
8,5,2020-07-01,Missing,Swine influenza virus,0,Missing,Fluid,NaN,NaN,PCR,Spain,Meat,4
9,5,2020-07-01,Missing,Swine influenza virus,1,Missing,Fluid,NaN,NaN,PCR,Spain,Meat,4


In [28]:
# PigAge mapping
PigAgeMapping = {

    '6.0 Semanas': '6 Weeks',
    '7.0 Semanas': '7 Weeks',
    '11.0 Semanas': '11 Weeks',
    '12.0 Semanas': '12 Weeks',
    '16.0 Semanas': '16 Weeks',
    '10.0 Semanas': '10 Weeks',
    '8.0 Semanas': '8 Weeks',
    '35.0 Dias': '35 Days',
    '56.0 Dias': '56 Days',
    '42.0 Dias': '42 Days',
    '60.0 Dias': '60 Days',
    '4.0 Semanas': '4 Weeks',
    '17.0 Semanas': '17 Weeks',
    '5.0 Semanas': '5 Weeks',
    '90.0 Kg': '90 Kg',
    '23.0 Dias': '23 Days',
    '9.0 Semanas': '9 Weeks',
    '3.0 Meses': '3 Months',
    '4.0 Meses': '4 Months',
    '7.0 Meses': '7 Months',
    '30.0 Kg': '30 Kg',
    '50.0 Kg': '50 Kg',
    '100.0 Kg': '100 Kg',
    '30.0 Dias': '30 Days',
    '3.0 Semanas': '3 Weeks',
    '40.0 Kg': '40 Kg',
    '25.0 Dias': '25 Days',
    '80.0 Kg': '80 Kg',
    '21.0 Dias': '21 Days',
    '49.0 Dias': '49 Days',
    '5.0 Meses': '5 Months',
    '45.0 Dias': '45 Days',
    'nan Missing': 'Missing',  # Handle the 'nan Missing' value
    '31.0 Dias': '31 Days',
    '20.0 Kg': '20 Kg',
    '13.0 Semanas': '13 Weeks',
    '50.0 Dias': '50 Days',
    '70.0 Dias': '70 Days',
    '90.0 Dias': '90 Days',
    '110.0 Dias': '110 Days',
    'Missing  ': 'Missing',  # Handle the 'Missing  ' value
    '17.0 Kg': '17 Kg',
    '21.0 Kg': '21 Kg',
    '26.0 Kg': '26 Kg',
    '35.0 Kg': '35 Kg',
    '15.0 Semanas': '15 Weeks',
    '24.0 Semanas': '24 Weeks',
    '40.0 Dias': '40 Days',
    '67.0 Dias': '67 Days',
    '28.0 Dias': '28 Days',
    '33.0 Dias': '33 Days',
    '47.0 Dias': '47 Days',
    '18.0 Kg': '18 Kg',
    '41.0 Dias': '41 Days',
    '62.0 Dias': '62 Days',
    '175.0 Dias': '175 Days',
    '135.0 Dias': '135 Days',
    '2.0 Meses': '2 Months',
    '58.0 Dias': '58 Days',
    '6.0 Meses': '6 Months',
    '18.0 Semanas': '18 Weeks',
    '30.0 Semanas': '30 Weeks',
    '10.0 Meses': '10 Months',
    '14.0 Semanas': '14 Weeks',
    '8.0 Kg': '8 Kg',
    '12.0 Kg': '12 Kg',
    '110.0 Kg': '110 Kg',
    '70.0 Kg': '70 Kg',
    '63.0 Dias': '63 Days',
    '2.0 Semanas': '2 Weeks',
    '1.0 Semanas': '1 Week',
    '25.0 Semanas': '25 Weeks',
    '20.0 Semanas': '20 Weeks',
    '105.0 Dias': '105 Days',
    '75.0 Dias': '75 Days',
    '32.0 Dias': '32 Days',
    '8.0 Meses': '8 Months',
    '37.0 Dias': '37 Days',
    '52.0 Dias': '52 Days',
    '27.0 Semanas': '27 Weeks',
    '26.0 Semanas': '26 Weeks',
    '28.0 Semanas': '28 Weeks',
    '23.0 Semanas': '23 Weeks',
    '60.0 Kg': '60 Kg',
    '54.0 Dias': '54 Days',
    '140.0 Dias': '140 Days',
    '130.0 Dias': '130 Days',
    '12.0 Meses': '12 Months',
    '14.0 Meses': '14 Months',
    '16.0 Kg': '16 Kg'
}

# Map the 'Age' column using the English translation mapping
df['PigAge'] = df['Age'].map(PigAgeMapping).fillna('Missing')


In [29]:
import pandas as pd

# Define the mapping dictionary
age_to_pig_stage = {
    
    '':'Missing',
    'nan':'Missing',
    '6 Weeks': 'Weaning',
    '7 Weeks': 'Weaning',
    '11 Weeks': 'Finishing',
    '12 Weeks': 'Finishing',
    '16 Weeks': 'Finishing',
    '10 Weeks': 'Nursery',
    '8 Weeks': 'Nursery',
    '35 Days': 'Nursery',
    '56 Days': 'Nursery',
    '42 Days': 'Nursery',
    '60 Days': 'Nursery',
    '4 Weeks': 'Suckling',
    '17 Weeks': 'Finishing',
    '5 Weeks': 'Nursery',
    '90 Kg': 'Finishing',
    '23 Days': 'Suckling',
    '9 Weeks': 'Nursery',
    '3 Months': 'Finishing',
    '4 Months': 'Finishing',
    '7 Months': 'Finishing',
    '30 Kg': 'Finishing',
    '50 Kg': 'Finishing',
    '100 Kg': 'Finishing',
    '30 Days': 'Weaning',
    '3 Weeks': 'Suckling',
    '40 Kg': 'Finishing',
    '25 Days': 'Suckling',
    '80 Kg': 'Finishing',
    '21 Days': 'Suckling',
    '49 Days': 'Nursery',
    '5 Months': 'Finishing',
    '45 Days': 'Nursery',
    '31 Days': 'Weaning',
    '20 Kg': 'Finishing',
    '13 Weeks': 'Finishing',
    '50 Days': 'Nursery',
    '70 Days': 'Nursery',
    '90 Days': 'Finishing',
    '110 Days': 'Finishing',
    '17 Kg': 'Finishing',
    '21 Kg': 'Finishing',
    '26 Kg': 'Finishing',
    '35 Kg': 'Finishing',
    '15 Weeks': 'Finishing',
    '24 Weeks': 'Finishing',
    '40 Days': 'Finishing',
    '67 Days': 'Finishing',
    '28 Days': 'Nursery',
    '33 Days': 'Nursery',
    '47 Days': 'Nursery',
    '18 Kg': 'Finishing',
    '41 Days': 'Nursery',
    '62 Days': 'Nursery',
    '6 Months': 'Finishing',
    '18 Weeks': 'Finishing',
    '30 Weeks': 'Finishing',
    '10 Months': 'Nursery',
    '14 Weeks': 'Suckling',
    '8 Kg': 'Finishing',
    '12 Kg': 'Finishing',
    '110 Kg': 'Finishing',
    '70 Kg': 'Finishing',
    '63 Days': 'Nursery',
    '2 Weeks': 'Suckling',
    '1 Week': 'Suckling',
    '25 Weeks': 'Finishing',
    '20 Weeks': 'Finishing',
    '105 Days': 'Finishing',
    '75 Days': 'Nursery',
    '32 Days': 'Nursery',
    '8 Months': 'Finishing',
    '37 Days': 'Nursery',
    '52 Days': 'Nursery',
    '27 Weeks': 'Finishing',
    '26 Weeks': 'Finishing',
    '28 Weeks': 'Finishing',
    '23 Weeks': 'Finishing',
    '60 Kg': 'Finishing',
    '54 Days': 'Nursery',
    '140 Days': 'Finishing',
    '130 Days': 'Finishing',
    '12 Months': 'Finishing',
    '14 Months': 'Finishing',
    '16 Kg': 'Finishing',
}


# Map the "Age" column to pig stages using the dictionary
df['PigStage'] = df['PigAge'].map(age_to_pig_stage)

# Print unique values in the "PigStage" column
print(df['PigStage'].unique())


[nan 'Weaning' 'Finishing' 'Nursery' 'Suckling']


In [30]:
unique_farm_types = df['BreedType'].unique()
print(unique_farm_types)

[nan 'Finishing' 'Sow' 'Weaning' 'Boar' 'Gilt']


In [31]:



# Combine the results from both mappings (Farmtype and PigStages)
df['PigStage'] = df.apply(
    lambda x: x['FarmType'] if pd.notna(x['FarmType']) else (x['BreedType'] if pd.notna(x['BreedType']) else x['PigStage']), axis=1).fillna("Missing")
df = df.drop(['FarmType'], axis=1)
df = df.drop(['PigAge'], axis=1)
df = df.drop(['BreedType'], axis=1)
df = df.drop(['Age'], axis=1)


In [32]:
df.head(50)

,SampleNumber,Date,Province,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,1,2020-07-01,Missing,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,4,Missing
1,1,2020-07-01,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
2,2,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
3,2,2020-07-01,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
4,3,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
5,3,2020-07-01,Missing,Swine influenza virus,1,Tissue,PCR,Spain,Meat,4,Missing
6,4,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
7,4,2020-07-01,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
8,5,2020-07-01,Missing,Swine influenza virus,0,Fluid,PCR,Spain,Meat,4,Missing
9,5,2020-07-01,Missing,Swine influenza virus,1,Fluid,PCR,Spain,Meat,4,Missing


In [33]:
df.to_csv('output/PigLab4.csv', index=False)

In [34]:
barometer = df

In [35]:
import pandas as pd

# Load the dataset
file_path = "data/PigData/barometer_combined_2024-10-08.csv"
df = pd.read_csv(file_path)

# Filter data for LabReference 1 and assign to 'barometer'
barometer = df[df['Lab_Reference'] == 4]

# Drop unwanted columns
barometer = barometer.drop(columns=['Month', 'Year', 'AutonomousCommunity'])

barometer['Pathogen'] = barometer['Pathogen'].str.strip().str.replace(' ', '_', regex=False)


# Rename 'FlooredDate' to 'Date'
barometer = barometer.rename(columns={'FlooredDate': 'Date'})

# Display the cleaned DataFrame
barometer.head()

# Optionally, save to CSV
barometer.to_csv("barometer.csv", index=False)



In [36]:
barometer

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
3053,1,Missing,2020-07-01,Mycoplasma_hyopneumoniae,0,Tissue,PCR,Spain,Meat,4,Missing
3054,1,Missing,2020-07-01,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,4,Missing
3055,2,Missing,2020-07-01,Mycoplasma_hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
3056,2,Missing,2020-07-01,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,4,Missing
3057,3,Missing,2020-07-01,Mycoplasma_hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
...,...,...,...,...,...,...,...,...,...,...,...
8541,2074,Missing,2023-07-14,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,4,Missing
8542,2074,Missing,2023-07-14,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,4,Missing
8543,2074,Missing,2023-07-14,Swine_influenza_virus,1,Fluid,PCR,Spain,Meat,4,Missing
8544,2074,Missing,2023-07-14,Swine_influenza_virus,1,Fluid,PCR,Spain,Meat,4,Missing


### Step 02: Create an RDF graph and namespaces.

In [37]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [38]:
#For Pig Data frame RDF conversion
for index, row in barometer.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab4PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab4.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
turtle_data = g.serialize(format="turtle")
print(turtle_data)

@prefix LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#> .
@prefix decide: <http://www.purl.org/decide#> .
@prefix ncit: <http://purl.obolibrary.org/obo/NCIT_C25464> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

LHO:Lab4PigSample_3053 a LHO:PigSample ;
    LHO:hasBreed LHO:Meat ;
    LHO:hasCountry LHO:Spain ;
    LHO:hasDate LHO:2020-07-01 ;
    LHO:hasDiagnosticTest LHO:PCR ;
    LHO:hasPathogen LHO:Mycoplasma_hyopneumoniae ;
    LHO:hasProductionStages LHO:Missing ;
    LHO:hasProvince LHO:Missing ;
    LHO:hasResult LHO:0 ;
    LHO:hasSampleType LHO:Tissue ;
    rdfs:comment "An individual representing a sample from a Pig and Piglets."@en .

LHO:Lab4PigSample_3054 a LHO:PigSample ;
    LHO:hasBreed LHO:Meat ;
    LHO:hasCountry LHO:Spain ;
    LHO:hasDate LHO:2020-07-01 ;
    LHO:hasDiagnosticTest LHO:PCR ;
    LHO:hasPathogen LHO:Swine_influenza_virus ;
    LHO:hasProductionStages LHO:Missing ;
    LHO:hasProvince LHO:Missing ;
    LHO:hasResult LHO:0 ;
    

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame: 

In [39]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=N8fd06fd647594e308b1cff5fea4103ad (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology 

In [40]:

query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
  (strafter(str(?PigProductionStages), "#") AS ?ProductionStage)
  (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)

WHERE {
 {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  FILTER (?SampleType = LHO:Fluid)
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  FILTER (?PigProductionStages = LHO:Weaning)
  ?Sample LHO:hasResult ?SampleResult .
  FILTER (?SampleResult = LHO:1)
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  }
}
"""


# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "Sample", "Pathogen", "Breed", "SampleType", 
    "DiagnosticTest","PigProductionStage", "Province", "Country", "SampleResult"
])

# Display the dataframe
df


,Sample,Pathogen,Breed,SampleType,DiagnosticTest,PigProductionStage,Province,Country,SampleResult
0,Lab4PigSample_3143,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Segovia,Spain,1
1,Lab4PigSample_3144,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Segovia,Spain,1
2,Lab4PigSample_3233,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Alicante,Spain,1
3,Lab4PigSample_3234,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Alicante,Spain,1
4,Lab4PigSample_3236,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Alicante,Spain,1
...,...,...,...,...,...,...,...,...,...
218,Lab4PigSample_8470,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Missing,Spain,1
219,Lab4PigSample_8471,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Missing,Spain,1
220,Lab4PigSample_8472,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Missing,Spain,1
221,Lab4PigSample_8473,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Missing,Spain,1
